In [76]:
import numpy as np
import torch
import matplotlib.pyplot as plt

import gym

from stable_baselines3 import PPO

from stable_baselines3.ppo.policies import MlpPolicy as MlpPolicy_ppo

from stable_baselines3 import DQN
from stable_baselines3.dqn.policies import DQNPolicy

from stable_baselines3.common.evaluation import evaluate_policy

import axelrod

In [82]:
tt = axelrod.titfortat.TitForTat()

TypeError: strategy() missing 1 required positional argument: 'opponent'

In [7]:
env = gym.make('CartPole-v0')

model = PPO("MlpPolicy", env, verbose=0)
# model = DQN("MlpPolicy", env, verbose=0)

In [8]:
def evaluate(model, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

In [9]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/home/peter/.local/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:19.66 +/- 7.83


In [10]:
mean_reward_before_train = evaluate(model, num_episodes=100)

Mean reward: 21.75 Num episodes: 100


In [11]:
model.learn(total_timesteps=100000, log_interval=4)

In [70]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:200.00 +/- 0.00


In [104]:
class T4TEnv(gym.Env):
    
    def __init__(self, RPST):
        super(T4TEnv, self).__init__()
        
        self.action_space = gym.spaces.Discrete(2)
        
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(4,),dtype=np.int)
        
        # history takes the form
        # self_c, self_d, other_c, other_d
        self.history = np.array([0, 0, 0, 0])
        
        
        self.payoff_mat = np.empty((2,2), dtype=np.object)
        self.payoff_mat[0, 0] = [RPST[0], RPST[0]]
        self.payoff_mat[1, 1] = [RPST[1], RPST[1]]
        self.payoff_mat[0, 1] = [RPST[2], RPST[3]]
        self.payoff_mat[1, 0] = [RPST[3], RPST[2]]
        
        self.RPST = RPST
        
        self.episode_len = 100
        self.step_num = 0
        
    def PD_game(self, a1, a2):
        # action 1: cooperate
        # action 0: defect
        
        r = int(not a1)
        c = int(not a2)
        return self.payoff_mat[r, c]
        
    def step(self, action):
        # action 1: cooperate
        # action 0: defect
        self.step_num += 1
        
        observation = np.array([0, 0, 0, 0])
        
        if self.history[1] == 0:
            observation[2] = 1
        else:
            observation[3] = 1
        
        if action:
            observation[0] = 1
        else:
            observation[1] = 1
            
        self.history = observation
        
        
        if step_num > episode_len:
            done = True
        else:
            done = False
            
        return observation, reward, done, info
    

    
    def reset(self):
        
        self.history = np.array([0, 0, 0, 0])
        observation = self.history
        
        return observation
    
    def render(self, mode='human'):
        
        pass
        
    def close(self):
        
        pass
        
        

In [105]:
int(True)

1

In [112]:
tt = T4TEnv([3,1,0,5])
tt.PD_game(False, False)

[1, 1]

In [73]:
aa = gym.spaces.Box(low=0, high=1, shape=(4,),dtype=np.int)

In [74]:
aa.sample()


array([0, 1, 1, 1])